<a href="https://colab.research.google.com/github/m0hamedyahya/paper-assignment/blob/main/mohamed_yahya_20108757.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
path_train='/content/drive/MyDrive/data/train'
path_test='/content/drive/MyDrive/data/test'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout,MaxPooling2D
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from keras.optimizers import Adam
import glob as gb
from keras.applications.vgg16 import VGG16

In [3]:
code = {'Class (1)': 0, 'Class (2)': 1}

def getcode(n):
    for x, y in code.items():
        if n == y:
            return x

In [4]:
x_train = []
y_train = []
img_size = 220

for i in os.listdir(path_train):
    files = gb.glob(pathname=str(os.path.join(path_train, i, '*.jpg')))

    for j in tqdm(files):
        image = cv2.imread(j)
        image_array = cv2.resize(image, (img_size, img_size))
        x_train.append(list(image_array))
        y_train.append(code[i])

# Convert to numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)
print(x_train.shape)
print(y_train.shape)

100%|██████████| 52/52 [00:00<00:00, 269.58it/s]


(104, 220, 220, 3)
(104,)


In [5]:
x_test = []
y_test = []
img_size = 220
for i in os.listdir(path_test):
    files = gb.glob(pathname=str(os.path.join(path_test, i, '*.jpg')))

    for j in tqdm(files):
        image = cv2.imread(j)
        image_array = cv2.resize(image, (img_size, img_size))
        x_test.append(list(image_array))
        y_test.append(code[i])


# Convert to numpy arrays
x_test = np.array(x_test)
y_test = np.array(y_test)
print(x_test.shape)
print(y_test.shape)

100%|██████████| 12/12 [00:00<00:00, 235.59it/s]

(24, 220, 220, 3)
(24,)


In [6]:
model_vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=(img_size, img_size, 3))
model = Sequential()

for layer in model_vgg16.layers:
  model.add(layer)

for layer in model.layers:
  layer.trainable = False # Not trainable weights

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 220, 220, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 220, 220, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 110, 110, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 110, 110, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 110, 110, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 55, 55, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 55, 55, 256)       2

In [17]:
model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])


In [24]:
early_stopping_monitor = EarlyStopping(patience = 2)
model.fit(x_train,y_train,epochs = 5,batch_size = 32,callbacks = [early_stopping_monitor])

Epoch 1/5
4/4 [==============================] - ETA: 0s - loss: 0.0509 - accuracy: 0.9904 

4/4 [==============================] - 64s 15s/step - loss: 0.0509 - accuracy: 0.9904
Epoch 2/5
4/4 [==============================] - ETA: 0s - loss: 1.3746e-04 - accuracy: 1.0000 

4/4 [==============================] - 67s 16s/step - loss: 1.3746e-04 - accuracy: 1.0000
Epoch 3/5
4/4 [==============================] - ETA: 0s - loss: 0.0132 - accuracy: 0.9904 

4/4 [==============================] - 64s 15s/step - loss: 0.0132 - accuracy: 0.9904
Epoch 4/5
4/4 [==============================] - ETA: 0s - loss: 0.0359 - accuracy: 0.9904 

4/4 [==============================] - 67s 16s/step - loss: 0.0359 - accuracy: 0.9904
Epoch 5/5
4/4 [==============================] - ETA: 0s - loss: 0.2936 - accuracy: 0.9904 

4/4 [==============================] - 64s 15s/step - loss: 0.2936 - accuracy: 0.9904


In [25]:
model.evaluate(x_test,y_test.astype('float32'))

1/1 [==============================] - 14s 14s/step - loss: 1.2379e-22 - accuracy: 1.0000


[1.2378551627060998e-22, 1.0]